In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import math
from functools import partial
from open3d.t.geometry import TriangleMesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dataname = "/home/chris/Code/PointClouds/data/ply/CircularVentilationGrateExtraCleanedFull.ply"
pcd = o3d.io.read_point_cloud(dataname)

In [3]:
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

PointCloud with 2116800 points.

In [4]:
#Outlier removal

nn = 16
std_multiplier = 10

filtered_pcd = pcd.remove_statistical_outlier(nn,std_multiplier)
outliers = pcd.select_by_index(filtered_pcd[1], invert = True)
outliers.paint_uniform_color([1,0,0])
filtered_pcd = filtered_pcd[0]

#o3d.visualization.draw_geometries([filtered_pcd, outliers])

In [5]:
#Downsampling

voxel_size = 0.01
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)
print(f'number of points: {len(pcd_downsampled.points)}')
#o3d.visualization.draw_geometries([pcd_downsampled])

number of points: 2114266


In [6]:
#Extract normals

nn_distance = np.mean([pcd.compute_nearest_neighbor_distance()])
radius_normals = nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

pcd_downsampled.paint_uniform_color([0.6,0.6,0.6])
#o3d.visualization.draw_geometries([pcd_downsampled, outliers])

PointCloud with 2114266 points.

In [7]:
# Extracting and Setting Parameters

front =  [-0.47452876114542436, 0.57451207113849134, -0.66690204300328082]
lookat = [-6.3976792217838847, 20.927374714553928, 18.659758576873813]
up =  [-0.056918726368614558, -0.77607794684805009, -0.62806311705487861]
zoom = 0.69999999999999996

pcd = pcd_downsampled

#o3d.visualization.draw_geometries([pcd_downsampled], zoom=zoom, front=front, lookat=lookat, up=up)

In [8]:
# Multi-order Ransac

max_plane_idx = 7
pt_to_plane_dist = 0.4

segment_models = {}
segments = {}
main_surface_idx = 0
largest_surface_points = 0
rest = pcd

for i in range(max_plane_idx):
    print(f'Run {i}/{max_plane_idx} started. ', end='')
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=50000)
    segments[i] = rest.select_by_index(inliers)
    if len(segments[i].points) > largest_surface_points:
        largest_surface_points = len(segments[i].points) 
        main_surface_idx = i
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print('Done')

print('Largest surface found with segment idx', main_surface_idx)
print(segment_models)
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest],zoom=zoom,front=front,lookat=lookat,up=up)

Run 0/7 started. Done
Run 1/7 started. Done
Run 2/7 started. Done
Run 3/7 started. Done
Run 4/7 started. Done
Run 5/7 started. Done
Run 6/7 started. Done
Largest surface found with segment idx 0
{0: array([ 0.0014122 ,  0.00389904,  0.9999914 , -0.7127501 ]), 1: array([ 0.50693457, -0.26853869,  0.81908749, -0.35286652]), 2: array([ 0.00774276, -0.57590381,  0.8174808 , -0.41157356]), 3: array([0.49481545, 0.29783264, 0.81636596, 0.25001557]), 4: array([-0.50440628,  0.27509492,  0.81847241, -2.06597412]), 5: array([-0.01026386,  0.57815082,  0.81586535, -0.67651789]), 6: array([-0.49078087, -0.28902581,  0.82194782, -1.46974496])}


In [9]:
#Calculate angles between main surface and extrusions

angles_rad = {}
angles_deg = {}

for i in range(len(segment_models)):
    if i != main_surface_idx:
        dot_product = np.dot(segment_models[main_surface_idx][:3], segment_models[i][:3])
        angles_rad[i] = np.arccos(np.clip(dot_product, -1, 1))
        angles_deg[i] = angles_rad[i] * 180 / np.pi
        #print(angles_deg[i])

print(angles_deg)

{1: 35.04021128812852, 2: 35.3889766935182, 3: 35.093097546461465, 4: 35.03259010898571, 5: 35.10514538125417, 6: 34.90311655041561}


In [10]:
#Find plane-to-plane intersection lines (the lines defined by the 
#cross_product for directionality and the common_point for line placement)

intersection_lines = {}

for i in range(len(segment_models)):
    if i != main_surface_idx:
        cross_product = np.cross(segment_models[main_surface_idx][:3], segment_models[i][:3])
        if abs(abs(cross_product[2]) < 0.0001):
            print(cross_product)
            common_point = np.linalg.solve([segment_models[main_surface_idx][1:3], segment_models[i][1:3]],
                                    [-segment_models[main_surface_idx][-1], -segment_models[i][-1]])
        else:
            common_point = np.linalg.solve([segment_models[main_surface_idx][:2], segment_models[i][:2]],
                                    [-segment_models[main_surface_idx][-1], -segment_models[i][-1]])

        common_point = np.asarray([common_point[0], common_point[1], 0])
            
        intersection_lines[i] = [cross_product,common_point]

print(intersection_lines)


{1: [array([ 0.27173003,  0.5057735 , -0.00235579]), array([ 81.83118673, 153.16299787,   0.        ])], 2: [array([ 0.57908625,  0.00658824, -0.00084348]), array([488.5476462 ,   5.85363682,   0.        ])], 3: [array([-0.29464704,  0.49365833, -0.0015087 ]), array([-141.34977175,  233.99730794,    0.        ])], 4: [array([-0.2719013 , -0.50555779,  0.00235519]), array([ 79.83170099, 153.88719518,   0.        ])], 5: [array([-0.57496476, -0.01141594,  0.00085648]), array([478.04704978,   9.65686659,   0.        ])], 6: [array([ 0.29222813, -0.4919374 ,  0.00150541]), array([-140.64850438,  233.74331466,    0.        ])]}


In [11]:
# Find the anchor points of the intersections of the planes and visualize

anchor_points = {}
for i in range(len(segment_models)):
    if i != main_surface_idx:
        segment_center = segments[i].get_center()
        b_vec = [sc - il for sc, il in zip(segment_center, intersection_lines[i][1])]
        a_vec = intersection_lines[i][0]/np.linalg.norm(intersection_lines[i][0])
        p_vec = np.dot(a_vec,b_vec)/np.dot(a_vec,a_vec) * a_vec
        anchor_points[i] = intersection_lines[i][1] + p_vec

#print(anchor_points)

sphere_radius = 5  # Adjust the radius based on your scale
spheres = []

for point in anchor_points.values():
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
    sphere.translate(point)  # Move the sphere to the anchor point
    sphere.paint_uniform_color([1, 0, 0])  # Red color for visibility
    spheres.append(sphere)

# Visualize
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest]+spheres, zoom=zoom, front=front, lookat=lookat, up=up)

In [12]:
# Function to create a line set between two points
def create_line(start_point, end_point, color=[1, 0, 0]):
    points = [start_point, end_point]
    lines = [[0, 1]]
    colors = [color]
    
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

# Function to rotate a vector by a specified angle around a given axis
def rotate_vector(vector, axis, angle_degrees):
    angle_radians = np.radians(angle_degrees)
    rotation_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(axis * angle_radians)
    rotated_vector = np.dot(rotation_matrix, vector)
    return rotated_vector

# Parameters for line visualization
line_length = 10  # Length of the lines

# Lists to store direction vectors
direction_vectors = []

# Generate the red and rotated green lines based on the projection and surface normals
lines = []
for i in range(len(segment_models)):
    if i != main_surface_idx:
        # Get the intersection line direction
        intersection_direction = intersection_lines[i][0]
        intersection_direction /= np.linalg.norm(intersection_direction)  # Normalize
        
        # Find a vector that is perpendicular to the intersection line
        perpendicular_direction = np.cross(intersection_direction, [0, 0, 1])
        perpendicular_direction /= np.linalg.norm(perpendicular_direction)  # Normalize
        
        # Anchor point for the line
        anchor_point = anchor_points[i]
        
        # Get the center of the bend (center of the bent surface)
        bend_center = segments[i].get_center()
        
        # Project the bend center onto the perpendicular line
        b_vec = bend_center - anchor_point
        projection_length = np.dot(b_vec, perpendicular_direction)
        projected_point = anchor_point + projection_length * perpendicular_direction
        
        # Determine the direction towards the bend
        direction_to_bend = (projected_point - anchor_point) / np.linalg.norm(projected_point - anchor_point)
        
        # Determine the direction towards the main surface (opposite direction)
        direction_to_main_surface = -direction_to_bend
        
        # Save the direction vectors
        direction_vectors.append({
            'anchor_point': anchor_point,
            'direction_to_main_surface': direction_to_main_surface,
            'direction_to_bend': direction_to_bend
        })
        
        # Draw only the red line towards the main surface
        main_surface_end_point = anchor_point + line_length * direction_to_main_surface
        main_surface_line = create_line(anchor_point, main_surface_end_point, color=[1, 0, 0])  # Red for main surface direction
        lines.append(main_surface_line)
        
        # Now draw the reversed green line based on the normal of the bent surface
        bent_surface_normal = segment_models[i][:3]  # Get the normal of the bent surface
        bent_surface_normal /= np.linalg.norm(bent_surface_normal)  # Normalize the normal
        
        # Reverse the direction of the green line by flipping the normal vector
        bent_surface_normal *= -1
        
        # Find the axis of rotation (perpendicular to both the red and green lines)
        rotation_axis = np.cross(direction_to_main_surface, bent_surface_normal)
        rotation_axis /= np.linalg.norm(rotation_axis)  # Normalize the rotation axis
        
        # Rotate the green line by 90 degrees around the calculated axis
        rotated_green_direction = rotate_vector(bent_surface_normal, rotation_axis, 90)
        
        # Create the rotated green line at the anchor point
        green_end_point = anchor_point + line_length * rotated_green_direction
        green_line = create_line(anchor_point, green_end_point, color=[0, 1, 0])  # Green for bent surface normal
        lines.append(green_line)
        
        # Save the green direction vector
        direction_vectors[-1]['rotated_green_direction'] = rotated_green_direction

# Visualize the point cloud, spheres, and the lines
o3d.visualization.draw_geometries([pcd_downsampled] + lines, zoom=zoom, front=front, lookat=lookat, up=up)

In [13]:
# Callback test

pcd_tree = o3d.geometry.KDTreeFlann(pcd)
eig_val_dict = {}
sample_dist = 1
number_of_neighbors = 10
sampled_neighborhoods = {}
initial_eigen_agg = 0
forward = True
#eigen_threshold = 100000
#angle_threshold = 0.11
eigen_threshold = 100000
angle_threshold = 0.11
radius = 5

class CallbackState:
    def __init__(self, anchor_points, pcd_tree, 
                 eigen_threshold,number_of_neighbors, 
                 sample_dist, radius, intersection_lines, pcd, angle_threshold):

        self.pcd_tree=pcd_tree
        self.pcd_color = (0.6,0.6,0.6)
        self.counter = 0  # This parameter changes each time the function is called
        self.forward=True
        self.anchor_points=anchor_points
        self.eigen_threshold=eigen_threshold
        self.angle_threshold=angle_threshold
        self.number_of_neighbors=number_of_neighbors
        self.sample_dist=sample_dist
        self.radius=radius
        self.intersection_lines=intersection_lines
        self.direction_vectors = direction_vectors
        self.segment_models = segment_models
        self.pcd = pcd
        self.pcd.paint_uniform_color(self.pcd_color)
        self.sampled_neighborhoods={}
        self.coloring_sampled_neighborhoods={}
        self.initial_eigen_agg=0
        self.initial_angle_agg=0
        self.eig_val_dict={}
        self.plane_dir_dict={}
        self.it=0
        self.seg_id = 1
        self.pcd_center = pcd.get_center()
        self.eigen_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text('Eigen agg =', depth=4).to_legacy()
        self.angle_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text('Angle agg =', depth=4).to_legacy()
        self.line_geometry = []
        self.ellipsoid_geometry = None

    def toggle_ellipsoid(self, vis):
        if self.ellipsoid_geometry:
            vis.remove_geometry(self.ellipsoid_geometry, reset_bounding_box=False)
            self.ellipsoid_geometry = None
        else:
            if self.it-1 in self.sampled_neighborhoods:
                # if self.forward:
                #     sampled_anchor_point = self.anchor_points[self.seg_id] + ((self.it-1)*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
                # else:
                #     sampled_anchor_point = self.anchor_points[self.seg_id] - ((self.it-1)*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
                idx = self.sampled_neighborhoods[self.it-1]
                neighbor_coordinates = np.asarray(self.pcd.points)[idx[1:], :]

                mu = np.mean(neighbor_coordinates, axis=0)
                norm = neighbor_coordinates - mu
                cov = np.cov(norm.T)
                eig_val, eig_vec = np.linalg.eig(cov)
                sorted_idx = np.argsort(eig_val)[::-1]
                eig_val = eig_val[sorted_idx]
                eig_vec = eig_vec[:, sorted_idx]

                # Create a sphere mesh
                sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)

                # Scale the sphere to form an ellipsoid based on the eigenvalues
                scale_matrix = 2*np.diag(np.sqrt(eig_val))

                # Construct the 4x4 transformation matrix
                transformation = np.eye(4)
                transformation[:3, :3] = eig_vec @ scale_matrix
                transformation[:3, 3] = mu

                # Apply the transformation to the sphere
                ellipsoid = sphere.transform(transformation)

                # Set the color of the ellipsoid
                ellipsoid.paint_uniform_color([0.2, 0.7, 0.3])  

                self.ellipsoid_geometry = ellipsoid
                vis.add_geometry(self.ellipsoid_geometry, reset_bounding_box=False)
        vis.update_renderer()
    
    def create_cylinder_between_points(self, point1, point2, radius, color):
        cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=np.linalg.norm(point2 - point1))
        cylinder.paint_uniform_color(color)
        
        # Compute the direction vector and normalize it
        direction = (point2 - point1) / np.linalg.norm(point2 - point1)
        
        # Compute the rotation axis and angle
        z_axis = np.array([0, 0, 1])
        rotation_axis = np.cross(z_axis, direction)
        rotation_axis /= np.linalg.norm(rotation_axis)
        rotation_angle = np.arccos(np.dot(z_axis, direction))
        
        # Apply the rotation
        cylinder.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * rotation_angle), center=(0, 0, 0))
        
        # Translate to the start point
        center = (point2+point1)/2
        cylinder.translate(center)
        
        return cylinder
    
    def create_cone_at_point(self, start_point, end_point, radius, color):
        radius = radius*2
        cone_height = radius * 3  # Adjust the height of the cone relative to the radius of the cylinder
        cone = o3d.geometry.TriangleMesh.create_cone(radius=radius, height=cone_height)
        cone.paint_uniform_color(color)

        # Compute the direction vector and normalize it
        direction = (end_point - start_point) / np.linalg.norm(end_point - start_point)

        # Compute the rotation axis and angle
        z_axis = np.array([0, 0, 1])
        rotation_axis = np.cross(z_axis, direction)
        if np.linalg.norm(rotation_axis) > 0:
            rotation_axis /= np.linalg.norm(rotation_axis)
        rotation_angle = np.arccos(np.dot(z_axis, direction))

        # Apply the rotation
        cone.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * rotation_angle), center=(0, 0, 0))

        # Translate to the end point
        cone.translate(end_point - direction * cone_height / 2)  # Adjust position to align cone tip to end_point

        return cone
    
    def key_callback(self, vis):
        #print(self.it)
        if self.seg_id > 6:
            #print('No more segments')
            return
        if self.forward:
            sampled_anchor_point = self.anchor_points[self.seg_id] + (self.it*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
        else:
            sampled_anchor_point = self.anchor_points[self.seg_id] - (self.it*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
        [k, idx, _] = self.pcd_tree.search_radius_vector_3d(sampled_anchor_point, self.radius)
        self.sampled_neighborhoods[self.it] = idx
        [k_color, idx_color, _] = self.pcd_tree.search_radius_vector_3d(sampled_anchor_point, 5)
        self.coloring_sampled_neighborhoods[self.it] = idx_color
        if k < 8:
            if self.forward:
                print('Point cloud boundary reached. Turning backwards')
                np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                self.it = 0
                self.forward = False
                self.eig_val_dict.clear()
                self.sampled_neighborhoods.clear()
                self.coloring_sampled_neighborhoods.clear()
                return
            else:
                np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                self.forward = True
                print('Point cloud boundary reached. Moving on to next segment')
                self.seg_id += 1
                self.it = 0
                self.sampled_neighborhoods.clear()
                self.coloring_sampled_neighborhoods.clear()
                self.eig_val_dict.clear()
                return

        neighbor_coordinates = np.asarray(self.pcd.points)[idx[1:], :]

        mu = np.mean(neighbor_coordinates, axis=0)
        norm = neighbor_coordinates - mu
        cov = np.cov(norm.T)
        eig_val, eig_vec = np.linalg.eig(cov)
        sorted_idx = np.argsort(eig_val)[::-1]
        eig_val = eig_val[sorted_idx]
        eig_vec = eig_vec[:, sorted_idx]
        #print(eig_val)
        eig_val_norm = eig_val.copy()
        for z in range(len(eig_val)):
            eig_val_norm[z] = np.exp(eig_val[z])/np.sum(np.exp(eig_val))
        self.eig_val_dict[self.it] = eig_val_norm

        plane_dir = np.cross(eig_vec[:, 0], eig_vec[:, 1])
        #print(f'Plane dict: {self.plane_dir_dict}')
        if self.it == 0:
            self.plane_dir_dict[self.it] = plane_dir
        elif self.it > 0:
            previous_plane_dir = self.plane_dir_dict[self.it-1]
            if np.dot(plane_dir,previous_plane_dir)<0:
                plane_dir = -plane_dir
            self.plane_dir_dict[self.it] = plane_dir


        eigen_agg = 0
        angle_agg = 0
        if self.it > self.number_of_neighbors:
            for eigen_index in range(len(eig_val_norm)):
                for prev_iteration in range(self.number_of_neighbors):
                    #Calculate aggragate for the eigenvalues
                    eigen_diff = eig_val_norm[eigen_index] - self.eig_val_dict[self.it-(prev_iteration+1)][eigen_index]
                    eigen_agg += (eigen_diff/self.sample_dist)**2
                    
            for prev_iteration in range(self.number_of_neighbors):
                #Calculate aggragate for the planes of the ellipsoids
                dot_product = np.dot(plane_dir, self.plane_dir_dict[self.it-(prev_iteration+1)])
                norm_1 = np.linalg.norm(plane_dir)
                norm_2 = np.linalg.norm(self.plane_dir_dict[self.it-(prev_iteration+1)])
                plane_angle = np.arccos(np.clip(dot_product/(norm_1*norm_2), -1, 1))
                angle_agg += plane_angle
            angle_agg /= self.number_of_neighbors
            #print(angle_agg)
            #print(eigen_agg)
            if self.it == self.number_of_neighbors + 1:
                self.initial_eigen_agg = eigen_agg
                self.initial_angle_agg = angle_agg
                # if forward:
                #     edge_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f"Edge {self.seg_id}", depth=4).to_legacy()
                #     edge_text_mesh.paint_uniform_color((1, 0, 0))
                #     edge_text_location = self.anchor_points[self.seg_id]
                #     edge_text_mesh.transform([[0.3, 0, 0, edge_text_location[0]+5], [0, 0.3, 0, edge_text_location[1]+5], [0, 0, 0.3, edge_text_location[2]+3], [0, 0, 0, 1]])
                #     vis.add_geometry(edge_text_mesh)
            if eigen_agg > self.eigen_threshold or angle_agg > self.angle_threshold:
                if self.forward:
                    #print(f'broke edge moving forwards with initial angle_agg: {self.initial_angle_agg} and current angle_agg: {angle_agg}')
                    #print('')
                    np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                    self.it = 0
                    self.forward = False
                    self.eig_val_dict.clear()
                    self.plane_dir_dict.clear()
                    self.sampled_neighborhoods.clear()
                    self.coloring_sampled_neighborhoods.clear()
                    return
                else:
                    #print(f'broke edge moving backwards with initial eigen_agg: {self.initial_eigen_agg} and current eigen_agg: {eigen_agg}')
                    #print('')
                    np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                    self.it = 0
                    self.forward = True
                    self.eig_val_dict.clear()
                    self.plane_dir_dict.clear()
                    self.sampled_neighborhoods.clear()
                    self.coloring_sampled_neighborhoods.clear()
                    self.seg_id += 1
                    return
            else: 
                blue_color = plt.get_cmap("tab20")(0)
                if self.it > self.number_of_neighbors + 1:
                    np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                if self.ellipsoid_geometry:
                    self.toggle_ellipsoid(vis=vis)
                np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it][1:], :] = (0.2,0.3,0.8)
                scale_factor = 4  # Adjust this factor to scale the length of the eigenvectors appropriately for visualization
                for line in self.line_geometry:
                    vis.remove_geometry(line, reset_bounding_box=False)
                self.line_geometry.clear()
                for i in range(3):  # Assuming 3D, so we have three principal components
                    start_point = mu
                    end_point = mu + np.sqrt(eig_val_norm[i]) * eig_vec[:, i] * scale_factor
                    color = [i == 0, i == 1, i == 2]  # Color the lines RGB based on principal component order
                    if np.linalg.norm(end_point - start_point) < 0.3:
                        end_point+=0.1* eig_vec[:, i]
                        radius = 0.04
                    else:
                        radius = 0.07  # Adjust the radius for thickness

                    # Create cylinder and add to visualizer
                    cylinder = self.create_cylinder_between_points(start_point, end_point, radius, color)
                    vis.add_geometry(cylinder)
                    self.line_geometry.append(cylinder)

                        # Create cone at the end point and add to visualizer
                    cone = self.create_cone_at_point(start_point, end_point, radius, color)
                    vis.add_geometry(cone)
                    self.line_geometry.append(cone)
                # self.eigen_agg_text_mesh.clear()
                # self.eigen_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f'Eigen agg = {eigen_agg:.2f}', depth=2).to_legacy()
                # self.eigen_agg_text_mesh.paint_uniform_color((1, 0, 0))
                # self.eigen_agg_text_location = sampled_anchor_point + [10,10,3]
                # self.eigen_agg_text_mesh.transform([[0.2, 0, 0, self.eigen_agg_text_location[0]], [0, 0.2, 0, self.eigen_agg_text_location[1]], [0, 0, 0.2, self.eigen_agg_text_location[2]], [0, 0, 0, 1]])
                # vis.add_geometry(self.eigen_agg_text_mesh)

                # self.angle_agg_text_mesh.clear()
                # self.angle_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f'Angle agg = {angle_agg:.2f}', depth=2).to_legacy()
                # self.angle_agg_text_mesh.paint_uniform_color((1, 0, 0))
                # self.angle_agg_text_location = sampled_anchor_point + [10,5,3]
                # self.angle_agg_text_mesh.transform([[0.2, 0, 0, self.angle_agg_text_location[0]], [0, 0.2, 0, self.angle_agg_text_location[1]], [0, 0, 0.2, self.angle_agg_text_location[2]], [0, 0, 0, 1]])
                # vis.add_geometry(self.angle_agg_text_mesh)

                view_ctl = vis.get_view_control()
                lookat = sampled_anchor_point
                zoom = 0.080000000000000002
                front = [-0.024106890455448116,-0.57254772319971181,0.81951690799604338]
                up =  [0.014828165865396817,0.81946017828866602,0.57294427451208185]
                view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
                view_ctl.set_up(up)      # Set the up direction of the camera
                view_ctl.set_front(front)  # Set the front direction of the camera
                view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera
                vis.update_geometry(self.pcd)
            
        self.it += 1

visualization = o3d.visualization.VisualizerWithKeyCallback()
state = CallbackState(anchor_points=anchor_points,pcd_tree=pcd_tree,eigen_threshold=eigen_threshold,number_of_neighbors=number_of_neighbors,sample_dist=sample_dist,
                      radius=radius,intersection_lines=intersection_lines, pcd=pcd, angle_threshold=angle_threshold)  
visualization.create_window()
visualization.get_render_option().background_color = np.asarray([0.95, 0.95, 0.95])
visualization.add_geometry(pcd)
#view_ctl = visualization.get_view_control()
#view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
#view_ctl.set_up(up)      # Set the up direction of the camera
#view_ctl.set_front(front)  # Set the front direction of the camera
#view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera

visualization.register_key_callback(263, state.toggle_ellipsoid)
visualization.register_key_callback(262, state.key_callback)
visualization.run()
visualization.destroy_window()

In [17]:
# Arc length

pcd_tree = o3d.geometry.KDTreeFlann(pcd)
eig_val_dict = {}
sample_dist = 0.05
number_of_neighbors = 3
sampled_neighborhoods = {}
initial_eigen_agg = 0
forward = True
#eigen_threshold = 100000
#angle_threshold = 0.11
eigen_threshold = 100000
angle_threshold = 0.01
radius = 0.7

class CallbackState:
    def __init__(self, anchor_points, pcd_tree, 
                 eigen_threshold,number_of_neighbors, 
                 sample_dist, radius, intersection_lines, pcd, angle_threshold):

        self.pcd_tree=pcd_tree
        self.pcd_color = (0.6,0.6,0.6)
        self.counter = 0  # This parameter changes each time the function is called
        self.forward=True
        self.anchor_points=anchor_points
        self.eigen_threshold=eigen_threshold
        self.angle_threshold=angle_threshold
        self.number_of_neighbors=number_of_neighbors
        self.sample_dist=sample_dist
        self.radius=radius
        self.intersection_lines=intersection_lines
        self.direction_vectors = direction_vectors
        self.segment_models = segment_models
        self.pcd = pcd
        self.pcd.paint_uniform_color(self.pcd_color)
        self.sampled_neighborhoods={}
        self.coloring_sampled_neighborhoods={}
        self.initial_eigen_agg=0
        self.initial_angle_agg=0
        self.eig_val_dict={}
        self.plane_dir_dict={}
        self.distance_travelled = {i: {'forward': 0, 'backward': 0} for i in range(1, len(anchor_points) + 1)}
        self.it=0
        self.seg_id = 1
        self.pcd_center = pcd.get_center()
        self.eigen_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text('Eigen agg =', depth=4).to_legacy()
        self.angle_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text('Angle agg =', depth=4).to_legacy()
        self.line_geometry = []
        self.ellipsoid_geometry = None

    def toggle_ellipsoid(self, vis):
        if self.ellipsoid_geometry:
            vis.remove_geometry(self.ellipsoid_geometry, reset_bounding_box=False)
            self.ellipsoid_geometry = None
        else:
            if self.it-1 in self.sampled_neighborhoods:
                # if self.forward:
                #     sampled_anchor_point = self.anchor_points[self.seg_id] + ((self.it-1)*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
                # else:
                #     sampled_anchor_point = self.anchor_points[self.seg_id] - ((self.it-1)*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
                idx = self.sampled_neighborhoods[self.it-1]
                neighbor_coordinates = np.asarray(self.pcd.points)[idx[1:], :]

                mu = np.mean(neighbor_coordinates, axis=0)
                norm = neighbor_coordinates - mu
                cov = np.cov(norm.T)
                eig_val, eig_vec = np.linalg.eig(cov)
                sorted_idx = np.argsort(eig_val)[::-1]
                eig_val = eig_val[sorted_idx]
                eig_vec = eig_vec[:, sorted_idx]

                # Create a sphere mesh
                sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)

                # Scale the sphere to form an ellipsoid based on the eigenvalues
                scale_matrix = 2*np.diag(np.sqrt(eig_val))

                # Construct the 4x4 transformation matrix
                transformation = np.eye(4)
                transformation[:3, :3] = eig_vec @ scale_matrix
                transformation[:3, 3] = mu

                # Apply the transformation to the sphere
                ellipsoid = sphere.transform(transformation)

                # Set the color of the ellipsoid
                ellipsoid.paint_uniform_color([0.2, 0.7, 0.3])  

                self.ellipsoid_geometry = ellipsoid
                vis.add_geometry(self.ellipsoid_geometry, reset_bounding_box=False)
        vis.update_renderer()
    
    def create_cylinder_between_points(self, point1, point2, radius, color):
        cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=np.linalg.norm(point2 - point1))
        cylinder.paint_uniform_color(color)
        
        # Compute the direction vector and normalize it
        direction = (point2 - point1) / np.linalg.norm(point2 - point1)
        
        # Compute the rotation axis and angle
        z_axis = np.array([0, 0, 1])
        rotation_axis = np.cross(z_axis, direction)
        rotation_axis /= np.linalg.norm(rotation_axis)
        rotation_angle = np.arccos(np.dot(z_axis, direction))
        
        # Apply the rotation
        cylinder.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * rotation_angle), center=(0, 0, 0))
        
        # Translate to the start point
        center = (point2+point1)/2
        cylinder.translate(center)
        
        return cylinder
    
    def create_cone_at_point(self, start_point, end_point, radius, color):
        radius = radius*2
        cone_height = radius * 3  # Adjust the height of the cone relative to the radius of the cylinder
        cone = o3d.geometry.TriangleMesh.create_cone(radius=radius, height=cone_height)
        cone.paint_uniform_color(color)

        # Compute the direction vector and normalize it
        direction = (end_point - start_point) / np.linalg.norm(end_point - start_point)

        # Compute the rotation axis and angle
        z_axis = np.array([0, 0, 1])
        rotation_axis = np.cross(z_axis, direction)
        if np.linalg.norm(rotation_axis) > 0:
            rotation_axis /= np.linalg.norm(rotation_axis)
        rotation_angle = np.arccos(np.dot(z_axis, direction))

        # Apply the rotation
        cone.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * rotation_angle), center=(0, 0, 0))

        # Translate to the end point
        cone.translate(end_point - direction * cone_height / 2)  # Adjust position to align cone tip to end_point

        return cone

    def visualize_direction_lines(self, anchor_point):
        # Remove previous lines from the visualizer
        
        # Clear the previous line references
        self.line_geometry.clear()

        # Red line for direction_to_main_surface
        red_direction = self.direction_vectors[self.seg_id-1]['direction_to_main_surface']
        red_end_point = anchor_point + 10 * red_direction  # Adjust length of line
        red_line = create_line(anchor_point, red_end_point, color=[1, 0, 0])
        self.line_geometry.append(red_line)

        # Green line for rotated_green_direction
        green_direction = self.direction_vectors[self.seg_id-1]['rotated_green_direction']
        green_end_point = anchor_point + 10 * green_direction  # Adjust length of line
        green_line = create_line(anchor_point, green_end_point, color=[0, 1, 0])
        self.line_geometry.append(green_line)

        # Add the lines to the visualizer


        # Update the visualization
        # vis.update_geometry(self.pcd)
        # vis.update_renderer()

    def calculate_arc_length(self):
        """
        Calculate the arc lengths for all segments and print them.
        The arc is calculated using the bend point, anchor point, and surface point.
        """
        for seg_id in range(1, len(self.anchor_points) + 1):
            # Retrieve the total distances traveled in both directions
            forward_distance = self.distance_travelled[seg_id]['forward']
            backward_distance = self.distance_travelled[seg_id]['backward']
            
            # Calculate the Bend Point (moving forward from anchor point)
            bend_point = self.anchor_points[seg_id] + forward_distance * self.direction_vectors[seg_id-1]['rotated_green_direction']
            
            # Calculate the Surface Point (moving backward from anchor point)
            surface_point = self.anchor_points[seg_id] + backward_distance * self.direction_vectors[seg_id-1]['direction_to_main_surface']
            
            # The anchor point is already known
            anchor_point = self.anchor_points[seg_id]
            
            circumcenter, radius = self.circumcenter_3d(anchor_point, surface_point, bend_point)
            vector_surface = surface_point - circumcenter
            vector_bend = bend_point - circumcenter

            # Normalize the vectors
            vector_surface_normalized = vector_surface / np.linalg.norm(vector_surface)
            vector_bend_normalized = vector_bend / np.linalg.norm(vector_bend)

            # Compute the angle between the two vectors using the dot product
            dot_product = np.dot(vector_surface_normalized, vector_bend_normalized)
            angle_radians = np.arccos(np.clip(dot_product, -1.0, 1.0))  # Ensure the dot product stays within [-1, 1]

            # Arc length is the angle in radians multiplied by the radius of the circle
            arc_length = angle_radians * radius
            # Print the result for this segment
            print(f"Arc length for Segment {seg_id}: {arc_length:.2f}")
            print(f'Total distance for Segment {seg_id}: {forward_distance+backward_distance}')
        print(f'distances: {self.distance_travelled}')
        print(15*'-')
    
    def circumcenter_3d(self, a, b, c):
            """
            Calculate the circumcenter of a triangle formed by three points in 3D space.
            """
            # Convert the points into numpy arrays for vector operations
            a = np.array(a)
            b = np.array(b)
            c = np.array(c)

            # Vector from a to c and a to b
            ac = c - a
            ab = b - a

            # Cross product of ab and ac
            abXac = np.cross(ab, ac)

            # Calculate the vector from point 'a' to the circumcenter
            to_circumcenter = (np.cross(abXac, ab) * np.dot(ac, ac) + np.cross(ac, abXac) * np.dot(ab, ab)) / (2.0 * np.dot(abXac, abXac))

            # Circumcenter coordinates in 3D space
            circumcenter = a + to_circumcenter

            # Circumsphere radius (distance from circumcenter to any of the points, e.g., a)
            circumsphere_radius = np.linalg.norm(to_circumcenter)

            return circumcenter, circumsphere_radius
    
    def key_callback(self, vis):
        #print(self.it)
        if self.seg_id > 6:
            self.calculate_arc_length()
            return
        if self.forward:
            sampled_anchor_point = self.anchor_points[self.seg_id] + (self.it * self.sample_dist) * self.direction_vectors[self.seg_id-1]['rotated_green_direction']
        else:
            sampled_anchor_point = self.anchor_points[self.seg_id] + (self.it*self.sample_dist)*self.direction_vectors[self.seg_id-1]['direction_to_main_surface']
        [k, idx, _] = self.pcd_tree.search_radius_vector_3d(sampled_anchor_point, self.radius)
        self.sampled_neighborhoods[self.it] = idx
        [k_color, idx_color, _] = self.pcd_tree.search_radius_vector_3d(sampled_anchor_point, 5)
        self.coloring_sampled_neighborhoods[self.it] = idx_color
        if k < 8:
            if self.forward:
                print('Point cloud boundary reached. Turning backwards')
                np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                self.it = 0
                self.forward = False
                self.eig_val_dict.clear()
                self.sampled_neighborhoods.clear()
                self.coloring_sampled_neighborhoods.clear()
                return
            else:
                np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                self.forward = True
                print('Point cloud boundary reached. Moving on to next segment')
                self.seg_id += 1
                self.it = 0
                self.sampled_neighborhoods.clear()
                self.coloring_sampled_neighborhoods.clear()
                self.eig_val_dict.clear()
                return

        neighbor_coordinates = np.asarray(self.pcd.points)[idx[1:], :]

        mu = np.mean(neighbor_coordinates, axis=0)
        norm = neighbor_coordinates - mu
        cov = np.cov(norm.T)
        eig_val, eig_vec = np.linalg.eig(cov)
        sorted_idx = np.argsort(eig_val)[::-1]
        eig_val = eig_val[sorted_idx]
        eig_vec = eig_vec[:, sorted_idx]
        #print(eig_val)
        eig_val_norm = eig_val.copy()
        for z in range(len(eig_val)):
            eig_val_norm[z] = np.exp(eig_val[z])/np.sum(np.exp(eig_val))
        self.eig_val_dict[self.it] = eig_val_norm

        plane_dir = np.cross(eig_vec[:, 0], eig_vec[:, 1])
        #print(f'Plane dict: {self.plane_dir_dict}')
        if self.it == 0:
            self.plane_dir_dict[self.it] = plane_dir
        elif self.it > 0:
            previous_plane_dir = self.plane_dir_dict[self.it-1]
            if np.dot(plane_dir,previous_plane_dir)<0:
                plane_dir = -plane_dir
            self.plane_dir_dict[self.it] = plane_dir


        eigen_agg = 0
        angle_agg = 0
        if self.it > self.number_of_neighbors:
            for eigen_index in range(len(eig_val_norm)):
                for prev_iteration in range(self.number_of_neighbors):
                    #Calculate aggragate for the eigenvalues
                    eigen_diff = eig_val_norm[eigen_index] - self.eig_val_dict[self.it-(prev_iteration+1)][eigen_index]
                    eigen_agg += (eigen_diff/self.sample_dist)**2
                    
            for prev_iteration in range(self.number_of_neighbors):
                #Calculate aggragate for the planes of the ellipsoids
                dot_product = np.dot(plane_dir, self.plane_dir_dict[self.it-(prev_iteration+1)])
                norm_1 = np.linalg.norm(plane_dir)
                norm_2 = np.linalg.norm(self.plane_dir_dict[self.it-(prev_iteration+1)])
                plane_angle = np.arccos(np.clip(dot_product/(norm_1*norm_2), -1, 1))
                angle_agg += plane_angle
            angle_agg /= self.number_of_neighbors
            #print(angle_agg)
            #print(eigen_agg)
            if self.it == self.number_of_neighbors + 1:
                self.initial_eigen_agg = eigen_agg
                self.initial_angle_agg = angle_agg
                # if forward:
                #     edge_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f"Edge {self.seg_id}", depth=4).to_legacy()
                #     edge_text_mesh.paint_uniform_color((1, 0, 0))
                #     edge_text_location = self.anchor_points[self.seg_id]
                #     edge_text_mesh.transform([[0.3, 0, 0, edge_text_location[0]+5], [0, 0.3, 0, edge_text_location[1]+5], [0, 0, 0.3, edge_text_location[2]+3], [0, 0, 0, 1]])
                #     vis.add_geometry(edge_text_mesh)
            if eigen_agg > self.eigen_threshold or angle_agg <= self.angle_threshold:
                if self.forward:
                    #print(f'broke edge moving forwards with initial angle_agg: {self.initial_angle_agg} and current angle_agg: {angle_agg}')
                    #print('')
                    #np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                    self.distance_travelled[self.seg_id]['forward'] = self.it*self.sample_dist
                    self.it = 0
                    self.forward = False
                    self.eig_val_dict.clear()
                    self.plane_dir_dict.clear()
                    self.sampled_neighborhoods.clear()
                    self.coloring_sampled_neighborhoods.clear()
                    return
                else:
                    #print(f'broke edge moving backwards with initial eigen_agg: {self.initial_eigen_agg} and current eigen_agg: {eigen_agg}')
                    #print('')
                    #np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                    self.distance_travelled[self.seg_id]['backward'] = self.it*self.sample_dist
                    self.it = 0
                    self.forward = True
                    self.eig_val_dict.clear()
                    self.plane_dir_dict.clear()
                    self.sampled_neighborhoods.clear()
                    self.coloring_sampled_neighborhoods.clear()
                    self.seg_id += 1
                    return
            else: 
                blue_color = plt.get_cmap("tab20")(0)
                #if self.it > self.number_of_neighbors + 1:
                    #np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                if self.ellipsoid_geometry:
                    self.toggle_ellipsoid(vis=vis)
                np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it][1:], :] = (0.2,0.3,0.8)
                scale_factor = 4  # Adjust this factor to scale the length of the eigenvectors appropriately for visualization
                for line in self.line_geometry:
                    vis.remove_geometry(line, reset_bounding_box=False)
                self.visualize_direction_lines(sampled_anchor_point)
                for line in self.line_geometry:
                    vis.add_geometry(line)
                # self.line_geometry.clear()
                # for i in range(3):  # Assuming 3D, so we have three principal components
                #     start_point = mu
                #     end_point = mu + np.sqrt(eig_val_norm[i]) * eig_vec[:, i] * scale_factor
                #     color = [i == 0, i == 1, i == 2]  # Color the lines RGB based on principal component order
                #     if np.linalg.norm(end_point - start_point) < 0.3:
                #         end_point+=0.1* eig_vec[:, i]
                #         radius = 0.04
                #     else:
                #         radius = 0.07  # Adjust the radius for thickness

                #     # Create cylinder and add to visualizer
                #     cylinder = self.create_cylinder_between_points(start_point, end_point, radius, color)
                #     vis.add_geometry(cylinder)
                #     self.line_geometry.append(cylinder)

                #         # Create cone at the end point and add to visualizer
                #     cone = self.create_cone_at_point(start_point, end_point, radius, color)
                #     vis.add_geometry(cone)
                #     self.line_geometry.append(cone)
                self.eigen_agg_text_mesh.clear()
                self.eigen_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f'Eigen agg = {eigen_agg:.2f}', depth=2).to_legacy()
                self.eigen_agg_text_mesh.paint_uniform_color((1, 0, 0))
                self.eigen_agg_text_location = sampled_anchor_point + [10,10,3]
                self.eigen_agg_text_mesh.transform([[0.2, 0, 0, self.eigen_agg_text_location[0]], [0, 0.2, 0, self.eigen_agg_text_location[1]], [0, 0, 0.2, self.eigen_agg_text_location[2]], [0, 0, 0, 1]])
                vis.add_geometry(self.eigen_agg_text_mesh)

                self.angle_agg_text_mesh.clear()
                self.angle_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f'Angle agg = {angle_agg:.2f}', depth=2).to_legacy()
                self.angle_agg_text_mesh.paint_uniform_color((1, 0, 0))
                self.angle_agg_text_location = sampled_anchor_point + [10,5,3]
                self.angle_agg_text_mesh.transform([[0.2, 0, 0, self.angle_agg_text_location[0]], [0, 0.2, 0, self.angle_agg_text_location[1]], [0, 0, 0.2, self.angle_agg_text_location[2]], [0, 0, 0, 1]])
                vis.add_geometry(self.angle_agg_text_mesh)

                view_ctl = vis.get_view_control()
                lookat = sampled_anchor_point
                zoom = 0.080000000000000002
                front = [-0.024106890455448116,-0.57254772319971181,0.81951690799604338]
                up =  [0.014828165865396817,0.81946017828866602,0.57294427451208185]
                view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
                view_ctl.set_up(up)      # Set the up direction of the camera
                view_ctl.set_front(front)  # Set the front direction of the camera
                view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera
                vis.update_geometry(self.pcd)
            
        self.it += 1

visualization = o3d.visualization.VisualizerWithKeyCallback()
state = CallbackState(anchor_points=anchor_points,pcd_tree=pcd_tree,eigen_threshold=eigen_threshold,number_of_neighbors=number_of_neighbors,sample_dist=sample_dist,
                      radius=radius,intersection_lines=intersection_lines, pcd=pcd, angle_threshold=angle_threshold)  
visualization.create_window()
visualization.get_render_option().background_color = np.asarray([0.95, 0.95, 0.95])
visualization.add_geometry(pcd)
#view_ctl = visualization.get_view_control()
#view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
#view_ctl.set_up(up)      # Set the up direction of the camera
#view_ctl.set_front(front)  # Set the front direction of the camera
#view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera

visualization.register_key_callback(263, state.toggle_ellipsoid)
visualization.register_key_callback(262, state.key_callback)
visualization.run()
visualization.destroy_window()

Arc length for Segment 1: 0.46
Total distance for Segment 1: 0.45
Arc length for Segment 2: 1.25
Total distance for Segment 2: 1.2
Arc length for Segment 3: 0.41
Total distance for Segment 3: 0.4
Arc length for Segment 4: 1.60
Total distance for Segment 4: 1.5500000000000003
Arc length for Segment 5: 1.19
Total distance for Segment 5: 1.1500000000000001
Arc length for Segment 6: 1.30
Total distance for Segment 6: 1.25
distances: {1: {'forward': 0.25, 'backward': 0.2}, 2: {'forward': 1.0, 'backward': 0.2}, 3: {'forward': 0.2, 'backward': 0.2}, 4: {'forward': 0.35000000000000003, 'backward': 1.2000000000000002}, 5: {'forward': 0.2, 'backward': 0.9500000000000001}, 6: {'forward': 0.2, 'backward': 1.05}}
---------------
Arc length for Segment 1: 0.46
Total distance for Segment 1: 0.45
Arc length for Segment 2: 1.25
Total distance for Segment 2: 1.2
Arc length for Segment 3: 0.41
Total distance for Segment 3: 0.4
Arc length for Segment 4: 1.60
Total distance for Segment 4: 1.550000000000000

In [15]:
if len(direction_vectors) > 0:
    first_anchor_point = direction_vectors[0]['anchor_point']
    
    # Draw the red line (direction to main surface)
    first_red_direction = direction_vectors[0]['direction_to_main_surface']
    first_red_end_point = first_anchor_point + line_length * first_red_direction
    first_red_line = create_line(first_anchor_point, first_red_end_point, color=[1, 0, 0])  # Red for main surface
    
    # Draw the green line (rotated green direction)
    first_green_direction = direction_vectors[0]['rotated_green_direction']
    first_green_end_point = first_anchor_point + line_length * first_green_direction
    first_green_line = create_line(first_anchor_point, first_green_end_point, color=[0, 1, 0])  # Green for rotated direction
    
    # Visualize the point cloud and the first red and green direction vectors
    o3d.visualization.draw_geometries([pcd_downsampled, first_red_line, first_green_line], zoom=zoom, front=front, lookat=lookat, up=up)